In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import sncosmo
from astropy.cosmology import FlatLambdaCDM

project_root = Path.cwd().parent
print(f"Project root: {project_root}")

Project root: /Users/david/Code/msc


In [2]:
# User input: run folder name 
folder_name = input("Enter the run folder name: ").strip()
run_folder = project_root / "runs" / folder_name

params_path = run_folder / "sncosmo_fits.csv"

df = pd.read_csv(params_path)
print(f"Loaded {len(df)} object(s) from {params_path}")
df.head()

# c : colour paramter, + means redder
# x1 : stretch parameter, + means broader lightcurve
# redshift : calculated from sncosmo fit
# t0 : time of maximum brightness in B-band
# x0 : overall flux scaling parameter
# x1 : stretch parameter, + means broader lightcurve

Enter the run folder name: test5
Loaded 4999 object(s) from /Users/david/Code/msc/runs/test5/sncosmo_fits.csv


,ztf_id,redshift,ncall,ndof,chisq,t0,x0,x1,c
0,ZTF17aabtvsy,0.069222,180,19,59.986343,59863.790107,0.000801,7.057003,0.177649
1,ZTF17aabvong,0.034000,101,27,274.734226,60607.466384,0.003006,0.128825,0.093196
2,ZTF17aacldgo,0.072000,138,13,56.213870,59898.042263,0.000461,-7.289450,0.056623
3,ZTF17aadlxmv,0.062000,112,24,61.256641,58878.493539,0.000932,0.716532,0.153930
4,ZTF18aaaonon,0.071419,132,9,108.298936,59707.377239,0.000581,-2.302907,-0.032337


In [4]:
import numpy as np
import pandas as pd
from astropy.cosmology import FlatLambdaCDM


# Fixed nuisance parameters,  depend on host galaxy, need more info https://www.aanda.org/articles/aa/pdf/2014/08/aa23413-14.pdf
# mu = mB - M + alpha*x1 - beta*c
alpha = 0.14
beta  = 3.1
M     = -19.36

# cosmology for theoretical distance modulus
cosmo = FlatLambdaCDM(H0=70.0, Om0=0.3)

ztf_cleansed_path = project_root / "ztf_cleansed.csv"
ztf_df = pd.read_csv(ztf_cleansed_path)

# use host redshifts from tns
hostz_map = dict(zip(ztf_df["ZTFID"].astype(str), ztf_df["host_redshift"]))

processed_rows = []

for _, row in df.iterrows():
    ztf_id = str(row["ztf_id"])

    z = hostz_map.get(ztf_id, row.get("redshift", np.nan))
    if pd.isna(z) or (isinstance(z, str) and not z.strip()):
        z = row.get("redshift", np.nan)
    if pd.isna(z):
        print(f"{ztf_id}: missing redshift, skipping.")
        continue
    z = float(z)

    # Read fit parameters
    ncall = int(row["ncall"])
    ndof  = int(row["ndof"])
    chisq = float(row["chisq"])

    t0 = float(row["t0"])
    x0 = float(row["x0"])
    x1 = float(row["x1"])
    c  = float(row["c"])

    if x0 <= 0 or not np.isfinite(x0):
        print(f"{ztf_id}: invalid x0={x0}, skipping.")
        continue

    mB = -2.5 * np.log10(x0) + 10.635  # B peak mag

    # Observed distance modulus after standardization
    mu_obs = mB - M + alpha * x1 - beta * c

    # Theoretical distance modulus cosmology
    mu_th = float(cosmo.distmod(z).value)

    # Hubble residual
    resid = mu_obs - mu_th

    processed_rows.append({
        "ztf_id": ztf_id,
        "host_redshift": z,
        "ncall": ncall,
        "ndof": ndof,
        "chisq": chisq,
        "t0": t0,
        "x0": x0,
        "x1": x1,
        "c": c,
        "mB": mB,
        "mu_obs": mu_obs,
        "mu_th": mu_th,
        "resid": resid,
    })

    print(
        f"{ztf_id}: z={z:.5f}, mB={mB:.3f}, mu_obs={mu_obs:.3f}, "
        f"mu_th={mu_th:.3f}, resid={resid:.3f}"
    )

# Save results
output_df = pd.DataFrame(processed_rows)
output_path = run_folder / "distance_process.csv"
output_df.to_csv(output_path, index=False)
print(f"\nWrote processed results to {output_path}")


ZTF17aabtvsy: z=0.06922, mB=18.375, mu_obs=38.173, mu_th=37.471, resid=0.702
ZTF17aabvong: z=0.03400, mB=16.940, mu_obs=36.029, mu_th=35.872, resid=0.157
ZTF17aacldgo: z=0.07200, mB=18.975, mu_obs=37.139, mu_th=37.560, resid=-0.421
ZTF17aadlxmv: z=0.06200, mB=18.212, mu_obs=37.195, mu_th=37.220, resid=-0.025
ZTF18aaaonon: z=0.07142, mB=18.724, mu_obs=37.862, mu_th=37.542, resid=0.321
ZTF18aaaooqj: z=0.05066, mB=18.756, mu_obs=37.026, mu_th=36.764, resid=0.262
ZTF18aaaqexr: z=0.07020, mB=18.828, mu_obs=37.694, mu_th=37.503, resid=0.191
ZTF18aadlaxo: z=0.05295, mB=18.346, mu_obs=37.205, mu_th=36.864, resid=0.341
ZTF18aadxnul: z=0.06400, mB=18.611, mu_obs=37.833, mu_th=37.292, resid=0.540
ZTF18aadzfso: z=0.05000, mB=18.694, mu_obs=37.417, mu_th=36.735, resid=0.683
ZTF18aaeqjmc: z=0.04300, mB=17.309, mu_obs=36.209, mu_th=36.396, resid=-0.187
ZTF18aaermez: z=0.02285, mB=19.108, mu_obs=34.584, mu_th=34.990, resid=-0.406
ZTF18aafdigb: z=0.06300, mB=18.739, mu_obs=37.345, mu_th=37.257, resid=0

/Users/david/opt/anaconda3/envs/jupyter/lib/python3.11/site-packages/astropy/cosmology/flrw/base.py:1351: RuntimeWarning: divide by zero encountered in log10
  val = 5.0 * np.log10(abs(self.luminosity_distance(z).value)) + 25.0


ZTF22aaphdzl: z=0.06163, mB=18.632, mu_obs=37.274, mu_th=37.207, resid=0.068
ZTF22aaphemj: z=0.06227, mB=18.708, mu_obs=37.649, mu_th=37.230, resid=0.419
ZTF22aapmmtu: z=0.08400, mB=19.091, mu_obs=38.037, mu_th=37.913, resid=0.124
ZTF22aaptcfi: z=0.03968, mB=17.289, mu_obs=36.243, mu_th=36.216, resid=0.026
ZTF22aaqhirm: z=0.08700, mB=19.037, mu_obs=38.363, mu_th=37.994, resid=0.369
ZTF22aaqlfwe: z=0.11691, mB=19.239, mu_obs=38.954, mu_th=38.679, resid=0.276
ZTF22aaqrflg: z=0.07900, mB=18.259, mu_obs=37.589, mu_th=37.772, resid=-0.183
ZTF22aaqrszt: z=0.05300, mB=17.568, mu_obs=37.997, mu_th=36.866, resid=1.131
ZTF22aaqwdjw: z=0.08000, mB=18.606, mu_obs=37.142, mu_th=37.801, resid=-0.660
ZTF22aarlppe: z=0.06700, mB=18.913, mu_obs=37.624, mu_th=37.396, resid=0.228
ZTF22aarrbne: z=0.06200, mB=18.967, mu_obs=37.613, mu_th=37.220, resid=0.393
ZTF22aartmuz: z=0.06918, mB=18.960, mu_obs=37.561, mu_th=37.469, resid=0.091
ZTF22aarvtbb: z=0.10300, mB=19.074, mu_obs=38.768, mu_th=38.384, resid=0.3


Wrote processed results to /Users/david/Code/msc/runs/test5/distance_process.csv
